# Date Preprocessing

This preprocessing file should be utilized to format the data to a format that can be joined by date.
- The date column should be named "date"
- The date column should have the format in "yyyy-mm"

### Convention
- We keep this separated from the final pre-processed files. 
- We save the date preprocessed files as a csv in the data folder so we don't have to call Redshift (too expensive)

# Import Files

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from pathlib import Path

from datetime import datetime

import redshift_connector

# Settings

In [4]:
# path to where we store our preprocessed data
data_file_path = Path("../data")

# Redshift Connection

In [ ]:
conn = redshift_connector.connect(
    host='cspc-workgroup.783764604578.us-west-2.redshift-serverless.amazonaws.com',
    database='cspc5071-dsa',
    port=5439,
    user='python_user',
    password='Database123!' # Not best practice but who cares :))
 )
cursor = conn.cursor()

# Query to get the list of tables in the specified schema
query = f"""
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
AND table_type = 'BASE TABLE';
"""

cursor.execute(query)
tables = cursor.fetchall()

print(f"Tables in schema public:")
for table in tables:
    print(table[0]) # These are all the data we have in AWS redshift.

In [6]:
# Function to query to redshift: Returns a dataframe 
def querying_to_redshift(query, cursor):
    cursor.execute(query)
    df : pd.DataFrame = cursor.fetch_dataframe()

    return df

# Date Preprocessing for Natural Disasters

In [6]:
def expand_date_range(row):
    date_range = pd.date_range(row["start_date"], row["end_date"], freq="MS").strftime("%Y-%m")
    num_months = len(date_range)

    return pd.DataFrame({
        "name": row["name"],
        "disaster_type": row["disaster_type"],
        "date": date_range,
        "cpi_adjusted_cost": row["cpi_adjsuted_cost"] / num_months,
        "unadjusted_cost": row["unadjusted_cost"] / num_months,
        "deaths": row["deaths"] / num_months
    })

In [7]:
query = """SELECT * FROM "cspc5071-dsa"."public"."us_national_disaster";"""
df_us_disasters = querying_to_redshift(query, cursor)

copied_df = df_us_disasters.copy() # creating a deep copy so I don't have to fetch the database all the time

# Changing date in format of 'YYYYMMDD' to 'YYYY-MM'
df_us_disasters['start_date'] = pd.to_datetime(df_us_disasters['start_date'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m')
df_us_disasters['end_date'] = pd.to_datetime(df_us_disasters['end_date'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m')

df_us_disasters = pd.concat(df_us_disasters.apply(expand_date_range, axis=1).to_list(), ignore_index=True)

df_us_disasters.to_csv(f"{data_file_path}/date_organized_us_disasters.csv")  # save data

# Date Preprocessing for US Disease Outbreak

In [ ]:
query = """SELECT year, month, primary_mode, etiology, illnesses, 
food_vehicle, animal_type FROM "cspc5071-dsa"."public"."us_national_outbreaks";
"""
df_us_outbreaks = querying_to_redshift(query, cursor)

In [9]:
df_us_outbreaks['date'] = df_us_outbreaks['year'].astype(str) + '-' + df_us_outbreaks['month'].astype(str)
df_us_outbreaks['datetime'] = pd.to_datetime(df_us_outbreaks['date'], format='%Y-%m', errors='coerce')
df_us_outbreaks['date'] = df_us_outbreaks['datetime'].dt.strftime('%Y-%m')

In [10]:
df_us_outbreaks_grouped = df_us_outbreaks.groupby('date', as_index=False).agg(
    us_human_outbreaks_cnt = pd.NamedAgg(column='illnesses', aggfunc='count'),
    us_human_illnesses_cnt = pd.NamedAgg(column='illnesses', aggfunc='sum')
)

In [ ]:
df_us_outbreaks_grouped.to_csv(f"{data_file_path}/date_organized_us_diseases.csv")

# Date Preprocessing for Population Data

In [13]:
query = """SELECT * FROM "cspc5071-dsa"."public"."us_population";
"""
df_us_population = querying_to_redshift(query, cursor)

In [14]:
df_us_population['date'] = pd.to_datetime(df_us_population['date'].astype(str), format='%Y-%m-%d').dt.strftime('%Y-%m')

In [ ]:
df_us_population.to_csv(f"{data_file_path}/date_organized_us_population.csv")

# Date preprocessing for Price Data

In [17]:
query = """SELECT * FROM "average_egg_price";"""
df_egg_price = querying_to_redshift(query, cursor)

In [18]:
df_egg_price['date'] = pd.to_datetime(df_egg_price['observation_date']).dt.strftime('%Y-%m')

In [ ]:
df_egg_price = df_egg_price[['date', 'price_per_dozen']].copy()

In [ ]:
df_egg_price.to_csv(f"{data_file_path}/date_organized_egg_price_for_merge.csv")

# Date preprocessing for covid-19 data

In [23]:
query = """SELECT * FROM "cspc5071-dsa"."public"."covid_hospitalization";"""
df_covid = querying_to_redshift(query, cursor)

In [24]:
df_covid_us = df_covid[df_covid['entity'] == 'United States'].copy()
df_covid_us['date'] = ((pd.to_datetime(df_covid_us['day'])).dt.to_period('M')).dt.strftime('%Y-%m')

In [26]:
df_covid_us = df_covid_us.groupby('date', as_index=False)['daily_hospital_occupancy'].mean().rename(columns={'daily_hospital_occupancy':'avg_daily_hospitalized'})

In [ ]:
df_covid_us.to_csv(f"{data_file_path}/date_organized_us_covid.csv")